In [1]:
import transformers
from transformers import RobertaTokenizer
import re
from typing import Dict, List, Union
from dataclasses import dataclass

In [2]:
@dataclass
class ProcessedText:
    """
    Container for processed text data
    """

    cleaned_text: str
    sentences: List[str]
    num_sentences: int

In [3]:
class RoBERTaPreprocessor:
    """
    Housing all the relevant preprocessing steps for RoBERTa within this class - namely:
    1. Cleaning all text while preserving important punctuation and structure.
    2. Splitting text into stenctence while taking care of abbreviations.
    3. Tokenizing and encoding the text.
    """

    def __init__(self, max_length: int = 512) -> None:
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.max_length = max_length
        self.abbreviations = {"mr.", "mrs.", "dr.", "st.", "ave.", "prof."}

    def clean_text(self, text: str) -> str:
        """
        Cleaning all text while preserving important punctuation and structure.
        """

        ## Replace multiple newlines/spaces with single space
        text = re.sub(r"\n+", " ", text)
        text = re.sub(r"\s+", " ", text)

        ## Remove URLs and emails:
        text = re.sub(r"http\S+|www\.\S+", "", text)  # Remove URLs
        text = re.sub(r"\b[\w-]+@[\w-]+[.][\w-]+", "", text)  # Remove emails

        ## Repalce multiple white-spaces with a single space
        text = " ".join(text.split())

        ## Normalize dashes to hyphen
        text = text.replace("—", "-").replace("–", "-")

        ## Fix spacing around punctuation
        text = re.sub(r"\s+([.,!?;:])", r"\1", text)
        text = re.sub(r"\(\s+", "(", text)
        text = re.sub(r"\s+\)", ")", text)

        ## Additional cleaning:
        text = re.sub(r"[\u0080-\uFFFF]", "", text)  # Remove non-ASCII characters
        text = re.sub(r"\d+", "NUM", text)  # Replace numbers with "NUM"

        # Remove leading/trailing whitespace
        return text.strip()

    def text2sentences(self, text: str) -> List[str]:
        """
        Split the text into sentences while handling common abbreviations
        """
        sentences = []
        current = []

        words = text.split()  ## Splitting based on whitespaces

        ## Iterate through each word until a stop-character is found.
        for word in words:
            current.append(word)
            if word.lower() in self.abbreviations:
                continue
            if word.endswith((".", "!", "?")):
                sentences.append(" ".join(current))
                current = []

        ## To add the last uncompleted sentence is any:
        if len(current) > 0:
            sentences.append(" ".join(current))

        return sentences

    def process_text(self, text: str) -> ProcessedText:
        """
        Clean & split into sentences
        Returns:
            ProcessedText object with cleaned text and sentences
        """
        cleaned_text = self.clean_text(text)
        sentences = self.text2sentences(cleaned_text)

        return ProcessedText(
            cleaned_text=cleaned_text,
            sentences=sentences,
            num_sentences=len(sentences),
        )

    def encode_for_model(
        self,
        processed_text: ProcessedText,
        add_special_tokens: bool = True,
        truncation: bool = True,
        padding: str = "max_length",
    ) -> Dict[str, List[int]]:
        """
        Encode processed text for RoBERTa.
        """
        try:
            return self.tokenizer(
                processed_text.cleaned_text,
                add_special_tokens=add_special_tokens,
                max_length=self.max_length,
                padding=padding,
                truncation=truncation,
                return_attention_mask=True,
                return_tensors="pt",
            )
        except Exception as e:
            print(f"Error encoding text: {str(e)}")
            return None

    def batch_encode(self, texts: List[str], **kwargs) -> Dict[str, List[List[int]]]:
        """
        Process and encode a batch of texts.
        """
        processed_texts = [self.process_text(text).cleaned_text for text in texts]

        return self.tokenizer(
            processed_texts,
            max_length=self.max_length,
            padding=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
            **kwargs,
        )

In [ ]:
preprocessor = RoBERTaPreprocessor()

## Running the code above for test:

In [4]:
import os
from pathlib import Path, PosixPath
from dotenv import load_dotenv
import json
from tqdm.notebook import tqdm
from rich import print

In [6]:
ENV_Path = Path("../envs/n24.env")
load_dotenv(str(ENV_Path))
data_dir = Path(os.environ.get("DATASET_ROOT"))
os.listdir(data_dir)

['.DS_Store', 'imgs', 'news']

In [7]:
doc_dir = data_dir / "news"
with open(doc_dir / "nytimes_train.json", "r") as fp:
    news_list = json.load(fp)

key_list = []
for news_dict in tqdm(news_list):
    for key in news_dict.keys():
        if key not in key_list:
            key_list.append(key)

sample_text = news_dict["article"]

  0%|          | 0/48988 [00:00<?, ?it/s]

In [11]:
processed = preprocessor.process_text(sample_text)
print("Cleaned text:", processed.cleaned_text)
print("\nSentences:", processed.sentences)

Cleaned text: The visionary landscapist Joseph Elmer Yoakum (NUM-NUM) has been categorized as an outsider, 
self-taught or folk artist. Whichever: His place in the expanding canon of NUMth-century American art is assured, 
both for his achievement and influence. The latest evidence of Yoakum's originality is this enthralling exhibition,
among the largest ever devoted to his work. It features nearly NUM of the artist's delirious vistas of undulant 
hills, mountains and rock formations, variously striated, patterned and creviced, rendered in pale browns and 
pastels of colored pencil burnished to resemble watercolor. Defined by double outlines, Yoakum's geological 
elements have a curious autonomy: They heave, lean and push against one another, but they also evoke soft creased 
flesh, voluptuous but slightly abstracted. Sudden breaks in the terrain offer views of tiny trees, distorting space
and scale and intensifying the sexual undercurrent. A magnetic ambiguity prevails. In "Mt Horseback on Rockey Knob 
Range Near Chillicothe, Ohio" (NUM), tan flowing forms divide the scene like irregular columns. They could be 
landslides, or an alternate universe alive with writhing snakes and worms. Yoakum concocted his fantastical 
topographies in a storefront on the South Side of Chicago during the last decade of his life, after years spent 
traveling the world as an itinerant worker. Once discovered, his art exerted an essential influence on Chicago 
Imagists like Jim Nutt, Roger Brown and Christina Ramberg. Beginning in NUM, he had several exhibitions in Chicago;
one at the Whitney Museum in New York opened in late NUM, just weeks before he died on Christmas Day. ROBERTA SMITH

Sentences:
[
    'The visionary landscapist Joseph Elmer Yoakum (NUM-NUM) has been categorized as an outsider, self-taught or 
folk artist.',
    'Whichever: His place in the expanding canon of NUMth-century American art is assured, both for his achievement
and influence.',
    "The latest evidence of Yoakum's originality is this enthralling exhibition, among the largest ever devoted to 
his work.",
    "It features nearly NUM of the artist's delirious vistas of undulant hills, mountains and rock formations, 
variously striated, patterned and creviced, rendered in pale browns and pastels of colored pencil burnished to 
resemble watercolor.",
    "Defined by double outlines, Yoakum's geological elements have a curious autonomy: They heave, lean and push 
against one another, but they also evoke soft creased flesh, voluptuous but slightly abstracted.",
    'Sudden breaks in the terrain offer views of tiny trees, distorting space and scale and intensifying the sexual
undercurrent.',
    'A magnetic ambiguity prevails.',
    'In "Mt Horseback on Rockey Knob Range Near Chillicothe, Ohio" (NUM), tan flowing forms divide the scene like 
irregular columns.',
    'They could be landslides, or an alternate universe alive with writhing snakes and worms.',
    'Yoakum concocted his fantastical topographies in a storefront on the South Side of Chicago during the last 
decade of his life, after years spent traveling the world as an itinerant worker.',
    'Once discovered, his art exerted an essential influence on Chicago Imagists like Jim Nutt, Roger Brown and 
Christina Ramberg.',
    'Beginning in NUM, he had several exhibitions in Chicago; one at the Whitney Museum in New York opened in late 
NUM, just weeks before he died on Christmas Day.',
    'ROBERTA SMITH'
]

In [12]:
## Encode fot model using processted text:
encoded = preprocessor.encode_for_model(processed)

In [15]:
tokens = preprocessor.tokenizer.convert_ids_to_tokens(encoded["input_ids"][0])
print("\nTokens: ", tokens[:10], "...")

Tokens: 
['<s>', 'The', 'Ġvisionary', 'Ġlandsc', 'apist', 'ĠJoseph', 'ĠEl', 'mer', 'ĠY', 'oak']
...